In [ ]:
import cyautomation.cyschoolhouse as cysh

**Contents**

* [Daily](#Daily)
* [As Needed or Requested](#As-Needed-or-Requested)
* [Weekly](#Weekly)
* [Start of Year](#Start-of-Year)
* [End of Year](#End-of-Year)

## Daily

In [ ]:
## sync enrollment in DESSA sections
results1 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Math',
    destination_section = 'DESSA',
    enrollment_start_date = '2019-09-03',
    ACM_to_TL = True
)

results2 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Literacy',
    destination_section = 'DESSA',
    enrollment_start_date = '2019-09-03',
    ACM_to_TL = True
)

results3 = cysh.student_section.enrollment_sync(
    source_section = 'Coaching: Attendance',
    destination_section = 'DESSA',
    enrollment_start_date = '2019-09-03',
    ACM_to_TL = True
)

# theoretically redundant
results4 = cysh.student_section.enrollment_sync(
    source_section = 'SEL Check In Check Out',
    destination_section = 'DESSA',
    enrollment_start_date = '2019-09-03',
    ACM_to_TL = True
)

## sync enrollment in MIRI sections
results5 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Math',
    destination_section = 'Math Inventory',
    enrollment_start_date = '2019-09-03'
)

results6 = cysh.student_section.enrollment_sync(
    source_section = 'Tutoring: Literacy',
    destination_section = 'Reading Inventory',
    enrollment_start_date = '2019-09-03'
)

## As Needed or Requested

#### Section Creation

In [ ]:
import pandas as pd

df = pd.concat([
    cysh.chi_section_creation.academic_sections_to_create(
        start_date='2019-09-03', end_date='2020-06-30'),
    cysh.chi_section_creation.non_academic_sections_to_create(
        start_date='2019-09-03', end_date='2020-06-30'),
    cysh.chi_section_creation.MIRI_sections_to_create(
        start_date='2019-09-03', end_date='2020-06-30')
])

print('Sections to make:')
df['SectionName'].value_counts()

In [ ]:
## Requires salesforce 2FA
cysh.section_creation.create_all_sections(df)

Since the default section is '50 Acts of Greatness', you may need to delete any errant sections that were made:

In [ ]:
cysh.chi_section_creation.deactivate_all_sections(section_type='50 Acts of Greatness')

#### Add New Students to cyschoolhouse

In [ ]:
## Requires salesforce 2FA
cysh.student.upload_all(
    xlsx_dir = 'Z:/ChiPrivate/Chicago Data and Evaluation/SY19', 
    xlsx_name = 'SY19 New Students for cyschoolhouse.xlsx', 
    enrollment_date='8/3/2018',
)

In [ ]:
#from time import sleep
#sleep(60)
cysh.student.update_student_External_Id()

#### Indicator Area Enrollment

In [ ]:
# load students to be assigned
df = cysh.chi_ia_assignment.get_ia_to_assign()
print(len(df), 'student IAs to enroll')

In [ ]:
# load browser
ia = cysh.chi_ia_assignment.IndicatorAreaEnrollment()

In [ ]:
# Enroll - Requires salesforce 2FA
ia.enroll_all_students(max_errors=100)

## Weekly

In [ ]:
service_trackers = cysh.WeeklyServiceTracker()
service_trackers.deploy_all()

In [ ]:
# when new ACMs enrolled
cch_trackers = cysh.CoachingLog()
cch_trackers.update_all_acm_stdnt_validation_sheets()

In [ ]:
# when new ACMs enrolled
ldr_trackers = cysh.LeadershipTracker()
ldr_trackers.update_all_acm_stdnt_validation_sheets()

In [ ]:
# when new ACMs enrolled or when new students enrolled in attendance
att_trackers = cysh.AttendanceTracker()
att_trackers.update_all_acm_stdnt_validation_sheets()

In [ ]:
## ToT Audit -- could be a Power BI report, but would need ACM access
tot_audit = cysh.ToTAudit()
tot_audit.deploy_all()

In [ ]:
## Thrive Data Share
cysh.chi_thrive_datashare.run()

## Start of Year

#### Deploy Trackers

In [ ]:
# ldr_trackers = cysh.LeadershipTracker(test=True)
# ldr_trackers.deploy_all()

In [ ]:
# cch_trackers = cysh.CoachingLog()
# cch_trackers.deploy_all()

In [ ]:
att_trackers = cysh.AttendanceTracker()
att_trackers.deploy_all()

## End of Year

In [ ]:
## Export salesforce Objects

## Exit all student_sections

# Misc.

In [ ]:
# # Associate TLs with their ACMs' SEL sections

# section_df = cysh.get_object_df('Section__c')
# section_df = section_df.loc[section_df['Name'].str.contains('SEL')]

# staff_df = cysh.get_object_df('Staff__c', ['Id', 'Name', 'Role__c', 'Organization__c'])
# staff_df = staff_df.loc[staff_df['Role__c'].str.contains('Team Lead')==True]

# staff_section_df = cysh.get_object_df('Staff_Section__c')

# results = []
# for index, row in section_df.iterrows():
#     if len(staff_section_df.loc[staff_section_df['Section__c']==row['Id']]) > 1:
#         print(f"ERROR: Section: {row['Id']} already has more than 1 staff associated")
#         continue
#     staff = staff_df.loc[staff_df['Organization__c']==row['School__c'], 'Id']
#     if len(staff) != 1:
#         print(f"ERROR: 0 or many Team Leaders associated with school: {row['School__c']}")
#         continue
#     staff_section_dict = {}
#     staff_section_dict['Account__c'] = row['School__c']
#     staff_section_dict['Section__c'] = row['Id']
#     staff_section_dict['Staff__c'] = staff.values[0]
#     staff_section_dict['Intervention_Primary_Staff__c'] = False
    
#     result = cysh.sf.Staff_Section__c.create(staff_section_dict)
#     results.append(result)

# sch_ref_df = pd.read_excel(r'Z:\ChiPrivate\Chicago Data and Evaluation\SY19\SY19 School Reference.xlsx')
# 
# tracker.fill_all_coaching_log_acm_rollup(sch_ref_df.loc[sch_ref_df['Informal Name']=='Chicago Academy'])